In [1]:
import math
import torch
import gpytorch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from GPyEm import GPE_ensemble as GPE

import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
#from GPErks.gp.data.dataset import Dataset
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.means import LinearMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from torchmetrics import MeanSquaredError, R2Score
#from GPErks.gp.experiment import GPExperiment
#from GPErks.train.emulator import GPEmulator
#from GPErks.perks.inference import Inference
#from GPErks.train.early_stop import NoEarlyStoppingCriterion
#from GPErks.train.early_stop import (
#    GLEarlyStoppingCriterion,
#    PQEarlyStoppingCriterion,
#    UPEarlyStoppingCriterion,
#)
#from GPErks.train.early_stop import PkEarlyStoppingCriterion



%matplotlib inline
%load_ext autoreload
%autoreload 2

# set logger and enforce reproducibility
#from GPErks.log.logger import get_logger
#from GPErks.utils.random import set_seed
#log = get_logger()
seed = 7
#set_seed(seed)
from time import process_time 
import scipy
from scipy.optimize import minimize

In [2]:
mode_weights = pd.read_csv(r'/Users/pmzcwl/Library/CloudStorage/OneDrive-TheUniversityofNottingham/shared_simulations/modes_weights.csv',index_col=0,delim_whitespace=False,header=0)

mode_weights

#mode_weights=mode_weights.drop(15,axis=0)
#mode_weights=mode_weights.drop(14,axis=0)

meshes=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19']

x_labels1=pd.read_csv(r'/Users/pmzcwl/Library/CloudStorage/OneDrive-TheUniversityofNottingham/shared_simulations/EP_healthy/input/xlabels_EP.txt',delim_whitespace=True,header=None)
x_labels=x_labels1.values.flatten().tolist()+mode_weights.columns.tolist()

y_labels=pd.read_csv(r'/Users/pmzcwl/Library/CloudStorage/OneDrive-TheUniversityofNottingham/shared_simulations/EP_healthy/output/ylabels.txt',delim_whitespace=True,header=None)



all_input = []
all_output=[]
all_x=[]
for i in range(len(meshes)):
    val=meshes[i]
    
    inputData = pd.read_csv("/Users/pmzcwl/Library/CloudStorage/OneDrive-TheUniversityofNottingham/shared_simulations/EP_healthy/"+val+"/X_EP.txt",index_col=None,delim_whitespace=True,header=None).values
    outputData = pd.read_csv("/Users/pmzcwl/Library/CloudStorage/OneDrive-TheUniversityofNottingham/shared_simulations/EP_healthy/"+val+"/Y.txt",index_col=None,delim_whitespace=True,header=None).values
    modeweights = np.tile(mode_weights.iloc[i,:].values, (inputData.shape[0],1))
    input_modes = np.concatenate((inputData,modeweights),axis=1)
    all_x.append(torch.tensor(inputData))
    all_input.append(torch.tensor(input_modes))
    all_output.append(torch.tensor(outputData))
    print(val)
    print(np.max(outputData))
#all_input=pd.concat(all_input)
#all_output=pd.concat(all_output
#all_input.columns=x_labels
#all_output.columns=y_labels

01
406.703
02
317.407
03
332.914
04
309.14
05
277.849
06
296.377
07
355.546
08
283.103
09
391.145
10
439.316
11
348.01
12
292.465
13
301.222
14
325.678
15
320.459
16
297.968
17
317.709
18
297.346
19
312.492


In [3]:
train_input=[]
test_input = []
train_output=[]
test_output = []

train_input_modes=[]
test_input_modes = []
train_output_modes=[]
test_output_modes = []

for i in range(len(meshes)):

    X=all_x[i]
    y=all_output[i]
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=seed+i
    )
    train_input.append(X_train)
    test_input.append(X_test)
    train_output.append(y_train)
    test_output.append(y_test)
    
for i in range(len(meshes)):

    X=all_input[i]
    y=all_output[i]
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=seed+i
    )
    train_input_modes.append(X_train)
    test_input_modes.append(X_test)
    train_output_modes.append(y_train)
    test_output_modes.append(y_test)


# Emulator per mesh

In [ ]:
emulators=[]
for i in range(len(meshes)):
    emulators.append(GPE.ensemble(train_input[i],train_output[i],mean_func="linear",training_iter=1000))

0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0


In [ ]:
reps=1

In [ ]:
R2 = torch.zeros(len(meshes),2)
R2_std = torch.zeros(len(meshes),2)
for i in range(len(meshes)):
    meanR, stdR = emulators[i].R2_sample(test_input[i],test_output[i],n=1000)
    R2[i,:]=meanR
    R2_std[i,:] = stdR

In [ ]:
R2.mean(axis=1)

In [ ]:
R2_std

In [ ]:
plt.rcParams["figure.figsize"] = (6.85,3.3)
fontS=12


In [ ]:
fig, ax = plt.subplots()


ax.scatter(meshes,R2[:,0].detach().numpy())
plt.errorbar(meshes,R2[:,0].detach().numpy(),fmt='o',yerr=R2_std[:,0].detach().numpy())
plt.setp(ax.get_xticklabels(), rotation=60, ha="right",
         rotation_mode="anchor");

ax.scatter(meshes,R2[:,1].detach().numpy(),marker='^')
plt.errorbar(meshes,R2[:,1].detach().numpy(),fmt='^',yerr=R2_std[:,1].detach().numpy())
plt.setp(ax.get_xticklabels(), rotation=60, ha="right",
         rotation_mode="anchor");

plt.legend(('A_TAT','V_TAT'))
plt.xlabel('Mesh')
plt.ylabel('$R^2$')
plt.xticks(fontsize=fontS)
plt.yticks(fontsize=fontS)
plt.tight_layout()


In [ ]:
all_Si_first = []
all_Si_total = []

for m in range(len(meshes)):
    input_masked=pd.DataFrame(train_input[m])
    # Generate max and min bounds
    nDim =input_masked.shape[1]
    boundsMaxMin = []
    for i in range(nDim):
        boundsMaxMin.append([np.min(input_masked.iloc[:,i]),np.max(input_masked.iloc[:,i])])
        print(boundsMaxMin[i])
    
    ##### from SALib.sample import saltelli
    from SALib.sample import saltelli
    from SALib.analyze import sobol
    from SALib.test_functions import Ishigami
    import numpy as np
    
    # Define the model inputs
    problem = {
        'num_vars': nDim,
        'names': x_labels1,
        'bounds': boundsMaxMin
        #'bounds': [[-1, 1],
         #          [-1, 1],
          #         [-1, 1],
           #        [-1, 1],
            #       [-1, 1],
             #      [-1, 1],
              #     [-1, 1],
               #    [-1, 1],
                #   [-1,1]]
    }
    
    # Generate samples
    param_values = saltelli.sample(problem, 512)
    
    data1 = torch.tensor(param_values)
    #data1.columns = X_train.columns
    
    Ymean=emulators[m].predict_sample(data1,n=5)
    
    Si = [[],[]]
    nMod = train_output[m].shape[1]
    nDim = input_masked.shape[1]
    for j in range(Ymean.shape[1]):
        for i in range(nMod):
            Si[i].append(sobol.analyze(problem, Ymean[:,j,i].detach().numpy(), print_to_console=False,calc_second_order=True))
    
    
    Si_first=np.zeros((nDim,nMod))
    Si_total=np.zeros((nDim,nMod))
    
    for j in range(Ymean.shape[1]):
        for i in range(nMod):
            total_Si, first_Si, second_Si= Si[i][j].to_df()
            Si_first[:,i] +=  first_Si.iloc[:,0]/Ymean.shape[1]
            Si_total[:,i] +=  total_Si.iloc[:,0]/Ymean.shape[1]

    all_Si_first.append(Si_first)
    all_Si_total.append(Si_total)

In [ ]:
inputs = x_labels1.iloc[:,0]
outputs = y_labels.iloc[:,0]
fig, ax = plt.subplots()
im = ax.imshow(all_Si_first[0],cmap='magma_r',vmin=0,vmax=1)

ax.set_xticks(np.arange(len(outputs)),labels=outputs)
ax.set_yticks(np.arange(len(inputs)), labels=inputs)
plt.setp(ax.get_xticklabels(), rotation=60, ha="right",
         rotation_mode="anchor");

plt.colorbar(im,fraction=0.015, pad=0.04)

# Emulator trained with 17/18 meshes and evaluated on the left out mesh

In [ ]:
train_p=40
reps=1
R2_test = torch.zeros(1,reps,2)
R2_leftout= torch.zeros(1,reps,2)
for i in range(1):
    for j in range(reps):
        X=torch.cat(train_input_modes)[:,0:15]
        y=torch.cat(train_output_modes)
        X_train, X_test, y_train, y_test = train_test_split(
            X,
            y,
            train_size=760,
            random_state=j
        )
        emulator=GPE.ensemble(X_train,y_train,mean_func="linear",training_iter=1000)
        meanR, stdR = emulator.R2_sample(torch.cat(test_input_modes)[:,0:15],torch.cat(test_output_modes),1000)
        
        R2_test[i,j,:]+=meanR

In [ ]:
R2_test

In [ ]:
input_masked=pd.DataFrame(X_train)
# Generate max and min bounds
nDim =input_masked.shape[1]
boundsMaxMin = []
for i in range(nDim):
    boundsMaxMin.append([np.min(input_masked.iloc[:,i]),np.max(input_masked.iloc[:,i])])
    print(boundsMaxMin[i])

##### from SALib.sample import saltelli
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

# Define the model inputs
problem = {
    'num_vars': nDim,
    'names': x_labels[0:15],
    'bounds': boundsMaxMin
    #'bounds': [[-1, 1],
     #          [-1, 1],
      #         [-1, 1],
       #        [-1, 1],
        #       [-1, 1],
         #      [-1, 1],
          #     [-1, 1],
           #    [-1, 1],
            #   [-1,1]]
}

# Generate samples
param_values = saltelli.sample(problem, 512)

data1 = torch.tensor(param_values)
#data1.columns = X_train.columns

Ymean=emulator.predict_sample(data1,n=5)



In [ ]:
Si = [[],[]]
nMod = y_train.shape[1]
nDim = X_train.shape[1]
for j in range(Ymean.shape[1]):
    for i in range(nMod):
        Si[i].append(sobol.analyze(problem, Ymean[:,j,i].detach().numpy(), print_to_console=False,calc_second_order=True))


Si_first=np.zeros((nDim,nMod))
Si_total=np.zeros((nDim,nMod))

for j in range(Ymean.shape[1]):
    for i in range(nMod):
        total_Si, first_Si, second_Si= Si[i][j].to_df()
        Si_first[:,i] +=  first_Si.iloc[:,0]/Ymean.shape[1]
        Si_total[:,i] +=  total_Si.iloc[:,0]/Ymean.shape[1]

In [ ]:
inputs = x_labels[0:15]
outputs = y_labels.iloc[:,0]
fig, ax = plt.subplots()
im = ax.imshow(Si_first,cmap='magma_r',vmin=0,vmax=1)

ax.set_xticks(np.arange(len(outputs)),labels=outputs)
ax.set_yticks(np.arange(len(inputs)), labels=inputs)
plt.setp(ax.get_xticklabels(), rotation=60, ha="right",
         rotation_mode="anchor");

plt.colorbar(im,fraction=0.015, pad=0.04)

# Discrepancy

In [ ]:
import time

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score


In [ ]:
model = Pipeline(steps=[
    #('scaler', StandardScaler()),
    ('preprocessor', PolynomialFeatures(degree=1, include_bias=False,interaction_only=False)),
    ('lasso', LassoCV(n_alphas=1000,max_iter=10000))
])

In [ ]:
def m0_mat(y_test,emulators,x_test,output):

    m0=torch.zeros((y_test.shape[0],len(emulators)))
    for i in range(len(emulators)):
        m0[:,i]=(emulators[i].predict(x_test)[:,output]-y_train.mean(axis=0)[output])/y_train.std(axis=0)[output]


    return m0

In [ ]:
def proxy(a,y_train,m0,output):
    m_t = (m0-y_train.mean(axis=0))/y_train.std(axis=0)
    y_t = (y_train-y_train.mean(axis=0))/y_train.std(axis=0)
    a=torch.tensor(a)
    res = ((a*m_t-y_t)**2).mean(axis=0).detach().numpy()
    return res[output]

In [ ]:
discrep_emulators = []

reps=1
nn=[60]
R2=torch.zeros(7,len(nn),2,reps)
ISE=torch.zeros(7,len(nn),2,reps)
Ti=torch.zeros(7,len(nn),reps)

for num, n in enumerate(nn):
    for k in range(len(emulators)):
        emulators2=emulators.copy()
        emulators2.pop(k)
        print(len(emulators2))

        X_train = train_input[k]
        y_train = train_output[k]
        X_test = test_input[k]
        y_test = test_output[k]
        
        for i in range(reps):

            #b=np.random.choice(range(X_train.shape[0]),n,replace=False)
            
            X=X_train
            y=y_train 
            X_train1, X_test1, y_train1, y_test1 = train_test_split(
                X,
                y,
                train_size=n,
                random_state=i
            )

    
            a_d=torch.zeros((y_train1.shape[1],len(emulators2)))
            for j in range(y_train1.shape[1]):
                m0=m0_mat(y_train1,emulators2,X_train1,j)
                # fit to an order-3 polynomial data
                y_t=(y_train1[:,j]-y_train1.mean(axis=0)[j])/y_train1.std(axis=0)[j]
                model = model.fit(m0.detach().numpy(), y_t.detach().numpy())
                a_d[j]=torch.tensor(model.named_steps['lasso'].coef_)




            start = time.time()
            model_dc_lasso_learned=GPE.ensemble(X_train1,y_train1,mean_func="discrepancy_cohort",training_iter=500,ref_emulator=emulators2,a=a_d,a_indicator=True)
            discrep_emulators.append(model_dc_lasso_learned)
            R2temp,R2std=model_dc_lasso_learned.R2_sample(X_test,y_test,1000)


In [ ]:
discrep_all_Si_first = []
discrep_all_Si_total = []

for m in range(len(meshes)):
    input_masked=pd.DataFrame(train_input[m])
    # Generate max and min bounds
    nDim =input_masked.shape[1]
    boundsMaxMin = []
    for i in range(nDim):
        boundsMaxMin.append([np.min(input_masked.iloc[:,i]),np.max(input_masked.iloc[:,i])])
        print(boundsMaxMin[i])
    
    ##### from SALib.sample import saltelli
    from SALib.sample import saltelli
    from SALib.analyze import sobol
    from SALib.test_functions import Ishigami
    import numpy as np
    
    # Define the model inputs
    problem = {
        'num_vars': nDim,
        'names': x_labels1,
        'bounds': boundsMaxMin
        #'bounds': [[-1, 1],
         #          [-1, 1],
          #         [-1, 1],
           #        [-1, 1],
            #       [-1, 1],
             #      [-1, 1],
              #     [-1, 1],
               #    [-1, 1],
                #   [-1,1]]
    }
    
    # Generate samples
    param_values = saltelli.sample(problem, 512)
    
    data1 = torch.tensor(param_values)
    #data1.columns = X_train.columns
    
    Ymean=discrep_emulators[m].predict_sample(data1,n=5)
    
    Si = [[],[]]
    nMod = train_output[m].shape[1]
    nDim = input_masked.shape[1]
    for j in range(Ymean.shape[1]):
        for i in range(nMod):
            Si[i].append(sobol.analyze(problem, Ymean[:,j,i].detach().numpy(), print_to_console=False,calc_second_order=True))
    
    
    Si_first=np.zeros((nDim,nMod))
    Si_total=np.zeros((nDim,nMod))
    
    for j in range(Ymean.shape[1]):
        for i in range(nMod):
            total_Si, first_Si, second_Si= Si[i][j].to_df()
            Si_first[:,i] +=  first_Si.iloc[:,0]/Ymean.shape[1]
            Si_total[:,i] +=  total_Si.iloc[:,0]/Ymean.shape[1]

    discrep_all_Si_first.append(Si_first)
    discrep_all_Si_total.append(Si_total)

In [ ]:
inputs = x_labels1.iloc[:,0]
outputs = y_labels.iloc[:,0]
fig, ax = plt.subplots()
im = ax.imshow(discrep_all_Si_first[0],cmap='magma_r',vmin=0,vmax=1)

ax.set_xticks(np.arange(len(outputs)),labels=outputs)
ax.set_yticks(np.arange(len(inputs)), labels=inputs)
plt.setp(ax.get_xticklabels(), rotation=60, ha="right",
         rotation_mode="anchor");

plt.colorbar(im,fraction=0.015, pad=0.04)